In [1]:
import numpy as np
from sklearn.datasets import fetch_california_housing

import tensorflow as tf

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Save

In [6]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

from sklearn.preprocessing import StandardScaler

n_epochs = 1000
learning_rate = 0.01

scaled_housing_data_plus_bias = StandardScaler().fit_transform(housing_data_plus_bias)
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform((n+1, 1)), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = tf.gradients(mse, [theta])[0]
# training_op = tf.assign(theta, theta - learning_rate * gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
            save_path = saver.save(sess, "tmp/my_model_%d.ckpt" % epoch)
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "tmp/my_model_final.ckpt")

Epoch 0 MSE =  7.8652096
Epoch 100 MSE =  4.947092
Epoch 200 MSE =  4.8856263
Epoch 300 MSE =  4.865403
Epoch 400 MSE =  4.8508396
Epoch 500 MSE =  4.8398933
Epoch 600 MSE =  4.8316116
Epoch 700 MSE =  4.8253183
Epoch 800 MSE =  4.820511
Epoch 900 MSE =  4.8168206


In [7]:
ls tmp

checkpoint                             my_model_800.ckpt.index
my_model_600.ckpt.data-00000-of-00001  my_model_800.ckpt.meta
my_model_600.ckpt.index                my_model_900.ckpt.data-00000-of-00001
my_model_600.ckpt.meta                 my_model_900.ckpt.index
my_model_700.ckpt.data-00000-of-00001  my_model_900.ckpt.meta
my_model_700.ckpt.index                my_model_final.ckpt.data-00000-of-00001
my_model_700.ckpt.meta                 my_model_final.ckpt.index
my_model_800.ckpt.data-00000-of-00001  my_model_final.ckpt.meta


### Restore

In [10]:
import tensorflow as tf
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, "tmp/my_model_final.ckpt")
    print(best_theta)

INFO:tensorflow:Restoring parameters from tmp/my_model_final.ckpt
[[ 0.90121615]
 [ 0.88760513]
 [ 0.15369044]
 [-0.33006862]
 [ 0.339271  ]
 [ 0.00745481]
 [-0.04351827]
 [-0.59074444]
 [-0.56597096]]
